In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [0]:
import pandas as pd
import gensim
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-12-08 01:20:43--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.96.5
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.96.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  22.7MB/s    in 43s     

2019-12-08 01:21:27 (36.2 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [6]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import tensorflow as tf

In [0]:
vocab = set()

In [0]:
from tensorflow.keras import backend as K
#from keras import backend as K
def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    return K.pow(2.0, cross_entropy)

In [0]:
emailTokens = pd.read_pickle('drive/My Drive/Next Word Prediction/emailTokens')

In [11]:
emailTokens.head()

0                            [here, is, our, forecast]
1    [traveling, have, business, meeting, takes, th...
2                          [test, successful, way, go]
3    [randy, can, you, send, me, schedule, the, sal...
4                      [lets, shoot, for, tuesday, at]
Name: message_tokens, dtype: object

In [0]:
#emailsdf['message_tokens'].head()

In [0]:
import ast

In [0]:
# with open('vocab.txt','w') as f:
#     f.write(str(vocab))

In [0]:
with open('drive/My Drive/Next Word Prediction/vocab.txt','r') as f:
    vocab = ast.literal_eval(f.read())

In [0]:
#Convert vocab set to a dictionary
word_map = dict()
inverse_word_map = dict()
i = 1
for x in vocab:
    word_map[x] = i
    inverse_word_map[i] = x
    i+=1

In [0]:
word_map[''] = 0
inverse_word_map[0] = ''

In [0]:
vocab.add('')

In [0]:
seq_len = 5

In [0]:
import copy

In [0]:
from tqdm import tqdm

In [0]:
def makeSequences(test):
    sequences = list()
    number_sequence = list()
    w2v_sequence = list()

    for sentence in test:
        for i in range(seq_len):
            sentence.insert(0,'')
        for i in range(seq_len,len(sentence)):
            sequence = sentence[i-seq_len:i+1]
            sequences.append(sequence)
            seq_i = list()
            seq_w2v = list()
            for word in sequence:
                seq_i.append(word_map[word])
                if(word == ""):
                    seq_w2v.append(np.zeros(300))
                else:
                    seq_w2v.append(word2vec[word])
            number_sequence.append(seq_i)
            w2v_sequence.append(seq_w2v)
    return (sequences,number_sequence,w2v_sequence)

In [0]:
def makeSequencestest(test):
    sequences = list()
    number_sequence = list()
    w2v_sequence = list()
    sentences = list()
    for sentence in test:
        for i in range(seq_len):
            sentence.insert(0,'')
        for i in range(seq_len,len(sentence)):
            sequence = sentence[i-seq_len:i+1]
            sequences.append(sequence)
            seq_i = list()
            seq_w2v = list()
            sen = list()
            for word in sequence:
                seq_i.append(word_map[word])
                if(word == ""):
                    seq_w2v.append(np.zeros(300))
                else:
                    seq_w2v.append(word2vec[word])
                sen.append(word)
            number_sequence.append(seq_i)
            w2v_sequence.append(seq_w2v)
            sentences.append(sen)
    return (sequences,number_sequence,w2v_sequence,sentences)

In [0]:
#Removing long emails
emailTokens = [s for s in emailTokens if len(s) <= 50 and len(s)>1]

In [25]:
len(emailTokens)

164559

In [0]:
#Train-Test Split 50-25-25
train = emailTokens[0:int(0.5*len(emailTokens))]
val = emailTokens[int(0.5*len(emailTokens)):int(0.75*len(emailTokens))]
test = emailTokens[int(0.75*len(emailTokens)):]

In [27]:
len(train)

82279

In [28]:
len(val)

41140

In [29]:
len(test)

41140

In [0]:
from numpy import array
#from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
#from keras.utils import to_categorical
#from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
#from keras.models import Sequential
from tensorflow.keras.layers import Dense
#from keras.layers import Dense,CuDNNLSTM
#from tensorflow.keras.layers import LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
#from keras.layers import Embedding

In [31]:
len(vocab)

77826

In [0]:
def sequence_x_y(data):
    (sequences,number_sequence,w2v_sequence) = makeSequences(data)
    w2v_X = list()
    #w2v_y = list()
    y = list()
    for i in range(len(w2v_sequence)):
        w2v_X.append(w2v_sequence[i][:-1])
        #w2v_y.append(w2v_sequence[i][-1])
        y.append(to_categorical(number_sequence[i][-1], num_classes=vocab_size))
    return ([np.array(w2v_X).reshape((-1, seq_len, 300))],[np.array(y)])

In [0]:
def sequence_x_ytest(data):
    (sequences,number_sequence,w2v_sequence,sentences) = makeSequencestest(data)
    w2v_X = list()
    #w2v_y = list()
    y = list()
    for i in range(len(w2v_sequence)):
        w2v_X.append(w2v_sequence[i][:-1])
        #w2v_y.append(w2v_sequence[i][-1])
        y.append(to_categorical(number_sequence[i][-1], num_classes=vocab_size))
    return ([np.array(w2v_X).reshape((-1, seq_len, 300))],[np.array(y)],[sentences])

In [34]:
vocab_size = len(vocab)
vocab_size

77826

In [0]:
batch_size = 128
val_batch_size = 16

In [0]:
from tensorflow.keras.utils import Sequence
#from keras.utils import Sequence

In [0]:
class KerasBatchGenerator(Sequence):

    def __init__(self, data,vocab_size,batch_size):
        self.data = data
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.current_idx = 0
        self.batchNum=1
        self.w2v_X = None
        self.y = None
        self.content = None
    def generate(self):
        while True:
            #gc.collect()
            #if(self.batchNum==1):
                #print("1st Batch")
            self.w2v_X = None
            self.y = None
            self.content = None
            self.batchNum+=1
            if(self.current_idx+self.batch_size >= len(self.data)):
                #print("Data from", str(self.current_idx),"till end")
                self.content = copy.deepcopy(self.data[self.current_idx:])
                self.batchNum = 1
                self.current_idx = 0
                gc.collect()
                #print("Epoch Completed",str(self.batchNum))
            else:
                #print("Data from", str(self.current_idx),"to",str(self.current_idx+self.batch_size))
                self.content = copy.deepcopy(self.data[self.current_idx:self.current_idx+self.batch_size])
                self.current_idx+=self.batch_size
            
            (self.w2v_X,self.y) = sequence_x_y(self.content)
            yield (self.w2v_X.pop(), self.y.pop())

In [0]:
#Used for evaluation
class KerastestBatchGenerator(Sequence):

    def __init__(self, data,vocab_size,batch_size):
        self.data = data
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.current_idx = 0
        self.batchNum=1
        self.w2v_X = None
        self.y = None
        self.content = None
        self.sentences = None
    def generate(self):
        while True:
            #gc.collect()
            #if(self.batchNum==1):
                #print("1st Batch")
            self.w2v_X = None
            self.y = None
            self.content = None
            self.sentences = None
            self.batchNum+=1
            if(self.current_idx+self.batch_size >= len(self.data)):
                #print("Data from", str(self.current_idx),"till end")
                self.content = copy.deepcopy(self.data[self.current_idx:])
                self.batchNum = 1
                self.current_idx = 0
                gc.collect()
                #print("Epoch Completed",str(self.batchNum))
            else:
                #print("Data from", str(self.current_idx),"to",str(self.current_idx+self.batch_size))
                self.content = copy.deepcopy(self.data[self.current_idx:self.current_idx+self.batch_size])
                self.current_idx+=self.batch_size
            
            (self.w2v_X,self.y,self.sentences) = sequence_x_ytest(self.content)
            yield (self.w2v_X.pop(), self.y.pop(),self.sentences.pop())

In [0]:
train_generator = KerasBatchGenerator(train,vocab_size,batch_size)

In [0]:
#Using only a part of the data for validation
val_generator = KerasBatchGenerator(val[0:100],vocab_size,val_batch_size)

In [0]:
num_epochs=40

In [0]:
import tensorflow as tf
# from tensorflow.keras.layers import LSTM, Dense

In [43]:
#Define the model
model = Sequential()
model.add(CuDNNLSTM(50, input_shape=(seq_len, 300)))
model.add(Dense(vocab_size, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',perplexity])

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 50)                70400     
_________________________________________________________________
dense (Dense)                (None, 77826)             3969126   
Total params: 4,039,526
Trainable params: 4,039,526
Non-trainable params: 0
_________________________________________________________________


In [0]:
import math
import tqdm

In [0]:
#Train the model
hist = model.fit_generator(train_generator.generate(), math.ceil(len(train)/(batch_size)), epochs = num_epochs,
                              validation_data=val_generator.generate(),
                              validation_steps=math.ceil(len(val[0:100])/(val_batch_size)),verbose=1)

Epoch 1/10
643/643 [==============================] - 562s 873ms/step - loss: 7.3257 - acc: 0.0379 - perplexity: 935.7480 - val_loss: 7.2462 - val_acc: 0.0497 - val_perplexity: 1203.6311
Epoch 2/10
643/643 [==============================] - 544s 846ms/step - loss: 6.8347 - acc: 0.0469 - perplexity: 500.0984 - val_loss: 6.9720 - val_acc: 0.0754 - val_perplexity: 791.7641
Epoch 3/10
643/643 [==============================] - 534s 830ms/step - loss: 6.5552 - acc: 0.0706 - perplexity: 451.0654 - val_loss: 6.7170 - val_acc: 0.0842 - val_perplexity: 680.5971
Epoch 4/10
643/643 [==============================] - 531s 826ms/step - loss: 6.2855 - acc: 0.0974 - perplexity: 401.0895 - val_loss: 6.5202 - val_acc: 0.1030 - val_perplexity: 612.7736
Epoch 5/10
643/643 [==============================] - 526s 818ms/step - loss: 6.0536 - acc: 0.1159 - perplexity: 357.4807 - val_loss: 6.3485 - val_acc: 0.1094 - val_perplexity: 568.6833
Epoch 6/10
643/643 [==============================] - 522s 811ms/step

In [0]:
hist.history

{'acc': [0.038161583,
  0.047983527,
  0.07284822,
  0.09809936,
  0.11452181,
  0.12683383,
  0.14405634,
  0.1632115,
  0.17599446,
  0.18941191,
  0.19891116,
  0.20820485,
  0.21667576,
  0.22238381,
  0.22763723,
  0.23257342,
  0.23723483,
  0.2412578,
  0.24485579,
  0.24800804,
  0.251143,
  0.25409165,
  0.25645918,
  0.25870958,
  0.26101187,
  0.26320052,
  0.2653101,
  0.2671879,
  0.26912946,
  0.2708348,
  0.27242157,
  0.27390453,
  0.2753406,
  0.27680427,
  0.27802733,
  0.27929732,
  0.2806093,
  0.28181162,
  0.28311175,
  0.28420186],
 'loss': [7.316876920573543,
  6.829084748976582,
  6.5455171526033515,
  6.2782118812676915,
  6.051699852368849,
  5.856082385629648,
  5.673014378459329,
  5.5104227564797785,
  5.37353292849194,
  5.258293249985147,
  5.159916819194751,
  5.074276493564709,
  4.999546957992154,
  4.933962097996374,
  4.875933995936033,
  4.824516829723745,
  4.778735741880142,
  4.7376851728674145,
  4.700779586611394,
  4.667264661332943,
  4.6367

In [0]:
#Loading the saved model
# # dependencies = {
# #     'perplexity': perplexity
# # }
# model = tf.keras.models.load_model('drive/My Drive/Next Word Prediction/model_seq5.h5',custom_objects=dependencies)
# model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 50)                70400     
_________________________________________________________________
dense (Dense)                (None, 77826)             3969126   
Total params: 4,039,526
Trainable params: 4,039,526
Non-trainable params: 0
_________________________________________________________________


In [0]:
# #Load the model history
# from pickle import load
# with open('drive/My Drive/Next Word Prediction/trainHistory', 'rb') as handle:
#     history = load(handle)

Evaluate the Model on Testing data

In [0]:
batch_size = 16
testGenerator = KerastestBatchGenerator(test[0:1000],vocab_size,batch_size)

In [0]:
import heapq

In [0]:
test_columns = ["sentence","ground_truth"]
for i in range(1,11):
    test_columns.append("pred_"+str(i))
test_columns

['sentence',
 'ground_truth',
 'pred_1',
 'pred_2',
 'pred_3',
 'pred_4',
 'pred_5',
 'pred_6',
 'pred_7',
 'pred_8',
 'pred_9',
 'pred_10']

In [0]:
test_accuracy = 0
test_accuracy_top5 = 0
test_accuracy_top10 = 0
total = 0
import math
for i in range(0,math.ceil(len(test[0:1000])/(batch_size))):
    print("Batch",str(i+1),"out of ",str(math.ceil(len(test[0:1000])/(batch_size))))
    df_pred = pd.DataFrame(columns=test_columns)
    (tempx,tempy,sen) = next(testGenerator.generate())
    tempx = tempx.reshape(-1,seq_len,300)
    numRows = tempx.shape[0]
    test_pred = model.predict_on_batch(tempx)
    y_test = [word_map[item[-1]] for item in sen]
    for j in range(numRows):
        pp = np.argmax(test_pred[j])

        total+=1
        temp = (-test_pred[j]).argsort()[:10]
        #Calculate Accuracies
        if(y_test[j] in temp):
            test_accuracy_top10+=1
        if(y_test[j] in temp[0:5]):
            test_accuracy_top5+=1
            if(y_test[j] == temp[0]):
                test_accuracy+=1
        row = {}
        row['sentence'] = sen[j][:-1]
        row['ground_truth'] = inverse_word_map[y_test[j]]
        for k in range(len(temp)):
            row['pred_'+str(k+1)] = inverse_word_map[temp[k]]
        df_pred = df_pred.append(row , ignore_index=True)
    df_pred.to_csv("drive/My Drive/Next Word Prediction/predictions_batch_"+str(i)+".csv")
    print("Accuracy=",str(test_accuracy/total))
    print("Top5Accuaracy=",str(test_accuracy_top5/total))
    print("Top10Accuaracy=",str(test_accuracy_top10/total))
    print("total=",str(total))
        

Batch 1 out of  63
Accuracy= 0.18952618453865336
Top5Accuaracy= 0.32418952618453867
Top10Accuaracy= 0.39900249376558605
total= 401
Batch 2 out of  63
Accuracy= 0.17232704402515722
Top5Accuaracy= 0.31572327044025156
Top10Accuaracy= 0.389937106918239
total= 795
Batch 3 out of  63
Accuracy= 0.17757847533632287
Top5Accuaracy= 0.3237668161434978
Top10Accuaracy= 0.3919282511210762
total= 1115
Batch 4 out of  63
Accuracy= 0.1845042678923178
Top5Accuaracy= 0.3315824031516743
Top10Accuaracy= 0.40840446487196325
total= 1523
Batch 5 out of  63
Accuracy= 0.18245425188374598
Top5Accuaracy= 0.3240043057050592
Top10Accuaracy= 0.4015069967707212
total= 1858
Batch 6 out of  63
Accuracy= 0.19075937785910338
Top5Accuaracy= 0.33989021043000917
Top10Accuaracy= 0.41765782250686184
total= 2186
Batch 7 out of  63
Accuracy= 0.18948983505945532
Top5Accuaracy= 0.34023782125047947
Top10Accuaracy= 0.41503644035289605
total= 2607
Batch 8 out of  63
Accuracy= 0.1922043010752688
Top5Accuaracy= 0.34038978494623656
Top

In [0]:
print(test_accuracy/total)
print(test_accuracy_top5/total)
print(test_accuracy_top10/total)

0.17643856269369293
0.3227238462182762
0.39659938018259483


LSTM with Sentiment Filtering

In [0]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

     |████████████████████████████████| 133kB 2.9MB/s 


In [0]:
analyser.polarity_scores("bad")

{'compound': -0.5423, 'neg': 1.0, 'neu': 0.0, 'pos': 0.0}

Testing Accuracy using Sentiment 

In [0]:
sentiment_map = dict()
for num,word in inverse_word_map.items():
  sentiment_map[num] = analyser.polarity_scores(word)  

In [0]:
batch_size = 16
testSentGenerator = KerastestBatchGenerator(test[0:1000],vocab_size,batch_size)

In [0]:
test_accuracy_sentiment = 0
test_accuracy_top5_sentiment = 0
test_accuracy_top10_sentiment = 0
total_sentiment = 0
import math
for i in range(0,math.ceil(len(test[0:1000])/(batch_size))):
    print("Batch",str(i+1),"out of ",str(math.ceil(len(test[0:1000])/(batch_size))))
    
    df_pred = pd.DataFrame(columns=test_columns)
    (tempx,tempy,sen) = next(testSentGenerator.generate())
    tempx = tempx.reshape(-1,seq_len,300)
    numRows = tempx.shape[0]
    test_pred = model.predict_on_batch(tempx)
    
    y_test = [word_map[item[-1]] for item in sen]
    for j in range(numRows):
        total_sentiment+=1
        prevSentiment = analyser.polarity_scores(''.join(sen[j][:-1]))
        
        top20 = (-test_pred[j]).argsort()[:20]
        valMap = {}
        for p in top20:
          vl=test_pred[j][p] + (2-abs(sentiment_map[p]['compound']-prevSentiment['compound']))
          valMap[p] = vl
        top10_sent = sorted(valMap.items(), key=lambda kv: kv[1], reverse=True)[0:10]
        temp = []
        for x in top10_sent:
          temp.append(x[0])
        #Calculate Accuracies
        if(y_test[j] in temp):
            test_accuracy_top10_sentiment+=1
        if(y_test[j] in temp[0:5]):
            test_accuracy_top5_sentiment+=1
            if(y_test[j] == temp[0]):
                test_accuracy_sentiment+=1
        row = {}
        row['sentence'] = sen[j][:-1]
        row['ground_truth'] = inverse_word_map[y_test[j]]
        for k in range(len(temp)):
            row['pred_'+str(k+1)] = inverse_word_map[temp[k]]
        df_pred = df_pred.append(row , ignore_index=True)
    df_pred.to_csv("drive/My Drive/Next Word Prediction/predictions_batch_"+str(i)+".csv")
    print("Accuracy=",str(test_accuracy_sentiment/total_sentiment))
    print("Top5Accuaracy=",str(test_accuracy_top5_sentiment/total_sentiment))
    print("Top10Accuaracy=",str(test_accuracy_top10_sentiment/total_sentiment))
    print("total=",str(total_sentiment))
        

Batch 1 out of  63
Accuracy= 0.18703241895261846
Top5Accuaracy= 0.32169576059850374
Top10Accuaracy= 0.40648379052369077
total= 401
Batch 2 out of  63
Accuracy= 0.1710691823899371
Top5Accuaracy= 0.3119496855345912
Top10Accuaracy= 0.3937106918238994
total= 795
Batch 3 out of  63
Accuracy= 0.17668161434977578
Top5Accuaracy= 0.31928251121076234
Top10Accuaracy= 0.39820627802690584
total= 1115
Batch 4 out of  63
Accuracy= 0.18319107025607353
Top5Accuaracy= 0.325016414970453
Top10Accuaracy= 0.41168745896257386
total= 1523
Batch 5 out of  63
Accuracy= 0.18083961248654468
Top5Accuaracy= 0.3153928955866523
Top10Accuaracy= 0.4041980624327234
total= 1858
Batch 6 out of  63
Accuracy= 0.18847209515096067
Top5Accuaracy= 0.32936870997255263
Top10Accuaracy= 0.4172003659652333
total= 2186
Batch 7 out of  63
Accuracy= 0.18757192174913695
Top5Accuaracy= 0.32988108937476024
Top10Accuaracy= 0.412734944380514
total= 2607
Batch 8 out of  63
Accuracy= 0.1905241935483871
Top5Accuaracy= 0.3299731182795699
Top10A

In [0]:
test_pred = model.predict(w2v_X_test.reshape(-1,seq_len,300))
test_pred.shape